In [33]:
import pandas as pd

In [34]:
countries = pd.read_csv("inputs/countries.csv")
population = pd.read_csv("inputs/population.csv")
regions = pd.read_csv("inputs/owid_regions.csv")
income_groups = pd.read_csv("inputs/income_groups.csv")
world = pd.read_csv("inputs/world_countries.csv")

# Emissions by country

In [35]:
nox = pd.read_csv("inputs/NOx_CEDS_emissions_by_country_2021_04_21.csv")
nox = nox.drop(columns=["em", "units"])
nox = nox.melt(id_vars=["country"],
               var_name="Year",
               value_name="nox")
nox["Year"] = nox["Year"].str.replace('X', '')

In [36]:
so2 = pd.read_csv("inputs/SO2_CEDS_emissions_by_country_2021_04_21.csv")
so2 = so2.drop(columns=["em", "units"])
so2 = so2.melt(id_vars=["country"],
               var_name="Year",
               value_name="so2")
so2["Year"] = so2["Year"].str.replace('X', '')

In [37]:
co = pd.read_csv("inputs/CO_CEDS_emissions_by_country_2021_04_21.csv")
co = co.drop(columns=["em", "units"])
co = co.melt(id_vars=["country"],
               var_name="Year",
               value_name="co")
co["Year"] = co["Year"].str.replace('X', '')

In [38]:
oc = pd.read_csv("inputs/OC_CEDS_emissions_by_country_2021_04_21.csv")
oc = oc.drop(columns=["em", "units"])
oc = oc.melt(id_vars=["country"],
               var_name="Year",
               value_name="oc")
oc["Year"] = oc["Year"].str.replace('X', '')

In [39]:
nmvoc = pd.read_csv("inputs/NMVOC_CEDS_emissions_by_country_2021_04_21.csv")
nmvoc = nmvoc.drop(columns=["em", "units"])
nmvoc = nmvoc.melt(id_vars=["country"],
               var_name="Year",
               value_name="nmvoc")
nmvoc["Year"] = nmvoc["Year"].str.replace('X', '')

In [40]:
bc = pd.read_csv("inputs/BC_CEDS_emissions_by_country_2021_04_21.csv")
bc = bc.drop(columns=["em", "units"])
bc = bc.melt(id_vars=["country"],
               var_name="Year",
               value_name="bc")
bc["Year"] = bc["Year"].str.replace('X', '')

In [41]:
nh3 = pd.read_csv("inputs/NH3_CEDS_emissions_by_country_2021_04_21.csv")
nh3 = nh3.drop(columns=["em", "units"])
nh3 = nh3.melt(id_vars=["country"],
               var_name="Year",
               value_name="nh3")
nh3["Year"] = nh3["Year"].str.replace('X', '')

In [42]:
combined = pd.merge(nox, so2)
combined = pd.merge(combined, co)
combined = pd.merge(combined, oc)
combined = pd.merge(combined, nmvoc)
combined = pd.merge(combined, bc)
combined = pd.merge(combined, nh3)

In [43]:
by_country = pd.merge(combined, countries)

In [44]:
by_country = by_country.drop(columns=["country"])

In [45]:
## Convert from thousand tonnes to tonnes
by_country["nox"] = by_country["nox"] * 1000
by_country["so2"] = by_country["so2"] * 1000
by_country["co"] = by_country["co"] * 1000
by_country["oc"] = by_country["oc"] * 1000
by_country["nmvoc"] = by_country["nmvoc"] * 1000
by_country["bc"] = by_country["bc"] * 1000
by_country["nh3"] = by_country["nh3"] * 1000

In [46]:
population = population.rename(columns={"country":"Entity",
                                       "year":"Year"})

population = population.drop(columns=["world_pop_share"])

In [47]:
by_country["Year"] = by_country["Year"].astype(int)
population["Year"] = population["Year"].astype(int)

In [48]:
by_country = pd.merge(by_country, population)

# Calculate totals by region

In [49]:
regions = pd.merge(by_country, regions, on=["Entity"], how="inner")

In [50]:
region_totals = regions.groupby(["Region","Year"])["nox", "so2", "co", "oc", "nmvoc", "bc", "nh3", "population"].sum().reset_index()

/var/folders/yw/t447kqtj65z8nt_wtmv6vj9c0000gn/T/ipykernel_6968/2143612132.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  region_totals = regions.groupby(["Region","Year"])["nox", "so2", "co", "oc", "nmvoc", "bc", "nh3", "population"].sum().reset_index()


In [51]:
region_totals = region_totals.rename(columns={"Region":"Entity"})

# Calculate global total

In [52]:
world = pd.merge(by_country, world, on=["Entity"], how="inner")

In [53]:
world_total = world.groupby(["Region","Year"])["nox", "so2", "co", "oc", "nmvoc", "bc", "nh3", "population"].sum().reset_index()
world_total = world_total.rename(columns={"Region":"Entity"})

/var/folders/yw/t447kqtj65z8nt_wtmv6vj9c0000gn/T/ipykernel_6968/1400265718.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  world_total = world.groupby(["Region","Year"])["nox", "so2", "co", "oc", "nmvoc", "bc", "nh3", "population"].sum().reset_index()


# Calculate totals by income group

In [54]:
income_groups = income_groups.rename(columns={"country":"Entity"})

In [55]:
income = pd.merge(by_country, income_groups, on=["Entity"], how="inner")

In [56]:
income_totals = income.groupby(["income_group","Year"])["nox", "so2", "co", "oc", "nmvoc", "bc", "nh3", "population"].sum().reset_index()

/var/folders/yw/t447kqtj65z8nt_wtmv6vj9c0000gn/T/ipykernel_6968/989755028.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  income_totals = income.groupby(["income_group","Year"])["nox", "so2", "co", "oc", "nmvoc", "bc", "nh3", "population"].sum().reset_index()


In [57]:
income_totals = income_totals.rename(columns={"income_group":"Entity"})

# Concat country, income groups and regions

In [58]:
by_country = pd.concat([by_country, income_totals, region_totals, world_total])

# Calculate per capita emissions

In [59]:
# Multiply each by 1000 to convert from tonnes per capita to kilograms per capita
by_country["nox_per_capita"] = by_country["nox"] / by_country["population"] * 1000
by_country["so2_per_capita"] = by_country["so2"] / by_country["population"] * 1000
by_country["co_per_capita"] = by_country["co"] / by_country["population"] * 1000
by_country["oc_per_capita"] = by_country["oc"] / by_country["population"] * 1000
by_country["nmvoc_per_capita"] = by_country["nmvoc"] / by_country["population"] * 1000
by_country["bc_per_capita"] = by_country["bc"] / by_country["population"] * 1000
by_country["nh3_per_capita"] = by_country["nh3"] / by_country["population"] * 1000

In [60]:
by_country = by_country.drop(columns=["population"])

In [61]:
by_country = by_country[ ["Entity"] + [ col for col in by_country.columns if col != "Entity" ] ]

In [62]:
by_country

,Entity,Year,nox,so2,co,oc,nmvoc,bc,nh3,nox_per_capita,so2_per_capita,co_per_capita,oc_per_capita,nmvoc_per_capita,bc_per_capita,nh3_per_capita
0,Aruba,1750,1.314951e-01,2.102971e-01,8.096439e+01,5.142993e+00,1.182583e+01,1.352049e+00,4.066145e+00,0.088430,0.141424,54.448143,3.458637,7.952813,0.909246,2.734462
1,Aruba,1760,1.403114e-01,2.236106e-01,8.609006e+01,5.468585e+00,1.258719e+01,1.437645e+00,4.375125e+00,0.087258,0.139061,53.538596,3.400861,7.827854,0.894058,2.720849
2,Aruba,1770,1.494873e-01,2.372893e-01,9.135637e+01,5.803109e+00,1.337248e+01,1.525588e+00,4.704933e+00,0.085912,0.136373,52.503659,3.335120,7.685331,0.876775,2.703984
3,Aruba,1780,1.589852e-01,2.512289e-01,9.672312e+01,6.144014e+00,1.417653e+01,1.615209e+00,5.056445e+00,0.084432,0.133419,51.366498,3.262886,7.528695,0.857785,2.685313
4,Aruba,1790,1.687518e-01,2.652916e-01,1.021373e+02,6.487930e+00,1.499246e+01,1.705622e+00,5.430453e+00,0.082843,0.130236,50.141023,3.185042,7.360070,0.837320,2.665907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,World,2015,1.025216e+08,8.181639e+07,5.621606e+08,1.372956e+07,1.457811e+08,5.937445e+06,5.912275e+07,13.902250,11.094557,76.230731,1.861771,19.768373,0.805136,8.017230
221,World,2016,9.994123e+07,7.696303e+07,5.442772e+08,1.374982e+07,1.450386e+08,5.832946e+06,5.969983e+07,13.399515,10.318736,72.973390,1.843493,19.445892,0.782046,8.004191
222,World,2017,9.902069e+07,7.336582e+07,5.373715e+08,1.375129e+07,1.452396e+08,5.827398e+06,6.033984e+07,13.128738,9.727267,71.247829,1.823226,19.256708,0.772630,8.000206
223,World,2018,9.827819e+07,7.310964e+07,5.321118e+08,1.366982e+07,1.442477e+08,5.787969e+06,6.087641e+07,12.888282,9.587657,69.781569,1.792671,18.916764,0.759039,7.983382


In [63]:
by_country.to_csv("output/Air pollution emissions (CEDS, 2022).csv", index=False)